In [ ]:
import os
import yaml
import random
import warnings
from tqdm import tqdm
from sklearn.metrics import f1_score

import numpy as np
import pandas as pd
from PIL import Image

from typing import NamedTuple, Tuple
from itertools import product

import matplotlib.pyplot as plt

In [ ]:
np.set_printoptions(precision=4)
warnings.filterwarnings('ignore')

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
import tensorflow as tf
tf.debugging.set_log_device_placement(False)

LABELS_PATH="labels.yaml"
IMAGES_BASE="/workspaces/motion/images/esp32/good"
BATCH_SIZE=10

# Taken from tensorflow GPU docs
# https://www.tensorflow.org/guide/gpu
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [ ]:
with open(LABELS_PATH) as f:
    label_map = yaml.load(f, Loader=yaml.SafeLoader)["labels"]
reverse_label_map = {v:k for k,v in label_map.items()}

examples = []
examples_labels = []
for dir,_,files in os.walk(IMAGES_BASE):
    for file in files:
        examples.append(np.asarray(Image.open(os.path.join(dir, file)).point(lambda x: 255 if x>0 else 0), dtype=np.float32).reshape(29, 40, 1))
        examples_labels.append(label_map[os.path.basename(dir)])
examples_numpy = np.asarray(examples)
examples_labels_numpy = np.asanyarray(examples_labels)

ds = tf.data.Dataset.from_tensor_slices((examples_numpy, examples_labels_numpy))
ds = ds.shuffle(buffer_size=2000, seed=1).batch(BATCH_SIZE)
display(ds)

val_ds = ds.take(10)
test_ds = ds.skip(10).take(10)
train_ds = ds.skip(20)

In [ ]:
discard_files = []
garbage_image_arrays = []
DISCARD_IMAGES_DIR="images/esp32/live/discard"
for dir,_,files in os.walk(DISCARD_IMAGES_DIR):
    for file in files:
        discard_files.append(os.path.join(dir, file))

for imgfile in discard_files:
    i = Image.open(imgfile).point(lambda x: 255 if x>0 else 0)
    garbage_img_array = np.asarray(i, dtype=np.uint8).reshape(29,40, 1)
    garbage_image_arrays.append(garbage_img_array)

all_garbage_images = np.array(garbage_image_arrays)

In [ ]:
class NetConfig(NamedTuple):
  conv_layers: int
  conv_per_layer: int
  conv_kernel_shape: Tuple[int]
  dropout1: float
  dropout2: float
  dense_size: int

def config_generator():
  conv_layers = [1,2,3]
  conv_per_layer = [2,4,6,8]
  conv_kernel_shape = [(3,3), (5,5)]
  dropout1 = [.5, .6, .7, .8, .9]
  dropout2 = [.5, .6, .7, .8, .9]
  dense_size = [256, 384, 512, 768, 1024]

  grid = [
    conv_layers,
    conv_per_layer,
    conv_kernel_shape,
    dropout1,
    dropout2,
    dense_size
  ]

  for c in product(*grid):
    yield NetConfig(
      conv_layers=c[0],
      conv_per_layer=c[1],
      conv_kernel_shape=c[2],
      dropout1=c[3],
      dropout2=c[4],
      dense_size=c[5]
    )

def generate_model(config: NetConfig):
  layers = []
  for i in range(config.conv_layers):
    layers.append(
      tf.keras.layers.Conv2D(config.conv_per_layer, config.conv_kernel_shape, padding='same', activation='relu')
    )
    layers.append(tf.keras.layers.MaxPool2D(strides=(2,2)))
    layers.append(tf.keras.layers.Flatten())
    layers.append(tf.keras.layers.Dropout(config.dropout1))
    layers.append(tf.keras.layers.Dense(config.dense_size, activation='relu'))
    layers.append(tf.keras.layers.Dropout(config.dropout2))
    layers.append(tf.keras.layers.Dense(8, activation='softmax'))
  model = tf.keras.models.Sequential(layers)

  model.compile(
    optimizer='adam',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

  history = model.fit(
      train_ds,
      epochs=30,
      validation_data=val_ds,
      verbose=0
  )
  return history


In [ ]:
configs = [c for c in config_generator()]
for c in tqdm(configs):
    h = generate_model(c)
    model = h.model

    real_classes = []
    pred_classes = []

    for batch in test_ds:
        preds = model.predict(batch[0], verbose=0)
        pred_classes.extend(tf.argmax(preds, 1).numpy())
        real_classes.extend(batch[1].numpy())

    s = f1_score(real_classes, pred_classes, average='weighted')

    preds = model.predict(all_garbage_images, verbose=0)
    # max_arg = np.argmax(preds, axis=1)
    max_value = np.amax(preds, axis=1)
    high_prob_crap_percent = (np.count_nonzero(max_value > .8)/len(max_value))*100
    print(c)
    print({k: v[29] for k,v in h.history.items()})
    print(f"Weighted F1 for test: {s}, High probability crap % {high_prob_crap_percent}")